## Intro

### Set of python scripts for:

- downloading the assembled genomes corresponding to a set of jc #'s
- blasting the assembled genomes against a set of specified blast databases for specific genes
- finding the best match (or lack thereof) to each genome for each gene, extracting the gene from the genome and saving it in a separate fasta file
- viewing the results of the blast searches (whether successful, and what the species of the best match was) in spreadsheets

### What you need to run this code:

- python3
- biopython
- rclone, with the shared Assembled_Genomes google drive configured as 'assembled_genomes'

## Functions and packages

#### Import packages

In [ ]:
import os
import glob
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

#### Run functions

In [ ]:
### Pulls the JC number out of a file name or other string
### Input: string
### output: JC #
def find_JC_number(mystring):
    import re
    try:
        return(re.search('JC-[0-9]+[0-9]+[0-9]+[0-9]+', mystring).group(0))
    except AttributeError:
        return(None)


### Pulls all the names of JC assemblies from the google drive via rclone into a list
### Input: name of rclone directory
### Output: list of all full paths to assembled genomes with "JC-" in the name
def get_JC_fasta_file_names(rclone_dir_name):
    import os
    genome_string = os.popen("rclone ls --include '*JC-*.fasta' "+rclone_dir_name+":").read()
    genome_list = [i.strip().split(None, 1)[1] for i in genome_string.split('\n') if i]
    return(genome_list)

### Get the full path of the corresponding genome assembly file on google drive from the JC number; 
### throw an error if there are no matches or more than one match
### Input: JC#, list from 'get_JC_fasta_file_names'
### Output: path to the assembled genome corresponding to the JC#
def get_JC_fasta_path_from_JC_num(JC_num, genome_list):
    matches=[]
    for fasta_file_path in genome_list:
        if JC_num in fasta_file_path:
            matches.append(fasta_file_path)
    if len(matches) == 1:
        return(matches[0])
    if len(matches) > 1:
        raise ValueError(JC_num+" has more than one match:\n"+"\n".join(matches))
    if len(matches) == 0:
        raise ValueError(JC_num+" has no matches in the genome list")

### Download an assembled genome from google drive/rclone using the full file path to the genome
### optional: specify a directory in which to download the genomes; otherwise will download to pwd
### Input: name of rclone directory, path to genome from 'get_JC_fasta_path_from_JC_num'
### Output: downloaded genome
def download_genome_from_drive(rclone_dir_name, path_to_genome, directory=""):
    from pathlib import Path
    fasta_name = path_to_genome.split("/")[-1]
    if directory == "":
        cp_dir=""
        fasta_location=fasta_name
    else:
        cp_dir="/"+directory
        fasta_location=cp_dir+"/"+fasta_name
    if Path(fasta_location).is_file():
        return(fasta_location)
    else:
        import os
        rclone_copy_cmd = "rclone copy "+'"'+rclone_dir_name+":"+path_to_genome+'"'+" ."+cp_dir
        #rclone_copy_cmd = "rclone copy "+'"'+rclone_dir_name+":"+path_to_genome+'"'+" ."
        os.system(rclone_copy_cmd)
        return(fasta_location)
    
### Combines multiple functions to download the assembled genomes corresponding to a JC #
### Inputs: JC #, output from 'get_JC_fasta_file_names' (list of assembled genomes)
### assumes the genomes you want are in 'assembled_genomes'
### Output: downloaded genome
def get_jc_fasta_all(JC_num, genome_list, directory="",rclone_dir_name="assembled_genomes"):
    path_to_genome = get_JC_fasta_path_from_JC_num(JC_num, genome_list)
    return(download_genome_from_drive(rclone_dir_name, path_to_genome, directory))

### Checks if a fasta file is already a blast database; otherwise makes in into a blast database
### Inputs: path to the fasta file in question, and whether it contains nucleotide or protein seqs ("nucl" or "prot")
### Note: currently just checks to see if ONE type of file extensino is present that hsoudl be for blast db
### could improve this to include all types
### Output: blast database or 'already a database' message
def make_blast_db(fasta_file, filetype, makenew=False):
    from pathlib import Path
    import os
    if filetype=="prot":
        check = ".pdb"
    else:
        check = ".ndb"
    if Path(fasta_file+check).is_file() and makenew==False:
        print(fasta_file, "already db")
        return(fasta_file)
    else:
        make_db_cmd = "makeblastdb -in "+fasta_file+" -dbtype "+filetype+" -parse_seqids"
        print(make_db_cmd)
        os.system(make_db_cmd) 
        return(fasta_file)
    
### Inputs: fasta file, blast db, blast db type ("prot" or "nucl"), evalue (defaults to 1e-4)
### Output: printed results of the blast search
### I don't remember why I made this but I had a good reason, I'm sure
### ASSUMES NUCLEOTIDE QUERY
def do_a_blast_and_print_results(fasta_query, database, database_type, evalue=1e-4):
    if "JC" in fasta_query:
        query_short_name = find_JC_number(fasta_query)
    else:
        query_short_name = fasta_query.rsplit(".",1)[0]
    if database_type == "prot":
        runtype = "blastx"
    elif database_type == "nucl":
        runtype = "blastn"
    else:
        print("no valid database type specified!")
    outname = "temp.blast.out"
    blast_statement = runtype+" -query "+fasta_query+" -db "+database+" -outfmt 6 -evalue "+str(evalue)+" -out "+outname
    #print(blast_statement)
    os.system(blast_statement)
    with open("temp.blast.out", "r") as infile:
        for line in infile:
            print(line.strip())
    os.system("rm "+outname)

### Inputs: fasta file, blast db, blast db type ("prot" or "nucl"), pretty database name, evalue (defaults to 1e-4)
### Outputs: blast results to folder 
def do_a_blast_to_folder(fasta_query, database, database_name, database_type, folder, evalue=1e-4):
    if "JC" in fasta_query:
        query_short_name = find_JC_number(fasta_query)
    else:
        query_short_name = fasta_query.rsplit(".",1)[0]
    if database_type == "prot":
        runtype = "blastx"
    elif database_type == "nucl":
        runtype = "blastn"
    else:
        print("no valid database type specified!")
    outname = folder+"/"+query_short_name+"_vs_"+database_name+"."+runtype
    blast_statement = runtype+" -query "+fasta_query+" -db "+database+" -outfmt 6 -evalue "+str(evalue)+" -out "+outname
    #print(blast_statement)
    os.system(blast_statement)

### Pulls out a gene from a genome using the scaffold/contig name and the start and stop position
### Inputs: fasta file with query genome, name of sequence with wanted gene, start and end indices of gene, 
### name for the new file to store the gene in
### outputs: nucleotide snippet in new fasta file
def pull_a_sequence(fasta_file, scaffold_name, start, end, newfilename):
    from Bio import SeqIO
    from Bio.SeqRecord import SeqRecord
    start = int(start)
    end = int(end)
    fragments_list = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        if record.id == scaffold_name:
            frag = record.seq[start:end]
            newrecord = SeqRecord(frag, record.id+"["+str(start)+":"+str(end)+"]", "", "")
            fragments_list.append(newrecord)
    SeqIO.write(fragments_list, newfilename, "fasta")

In [ ]:
### Creates a directory if it doesn't already exist
def new_dir(directory):
    import os
    if not os.path.exists(directory):
        os.makedirs(directory)

### Combines many of the above functions to download the 
### assembled genomes corresponding to a list of jc #'s in a text file
### Inputs: txt file with list of jc #s, place to download (default is a folder called 'jc_files')
### Note: this will check the specified directory to see if any are already downloaded
### Outputs: a directory full of the downloaded genomes that you want
def download_jcs_from_list(file_of_jc_nums,download_directory="jc_files", rclone_dir = "assembled_genomes"):
    jc_dict = {}
    new_dir(download_directory)
    dict_of_already_downloaded_files = {find_JC_number(i):i for i in glob.glob(download_directory+"/"+"*.fasta")}
    with open(file_of_jc_nums, "r") as infile:
        for i in infile:
            jc_num = i.strip()
            if jc_num in dict_of_already_downloaded_files:
                jc_dict[jc_num] = dict_of_already_downloaded_files[jc_num]
            else:
                jc_dict[jc_num] = get_jc_fasta_all(jc_num, genome_list, directory=download_directory,rclone_dir_name=rclone_dir)
    return(jc_dict)


## Preliminary steps for running blast searches (can do these steps without input JC #'s or query genomes)

### Get the list of assembled genomes in the google drive "Assembled Genomes" (this takes a little while to run)

In [ ]:
genome_list = get_JC_fasta_file_names("assembled_genomes")

Setup:
    - directory of dbs, with individual folder for each db that is labeled wtih a good name for that gene 
    - 

### Get a spreadsheet showing you all the databases of genes you have, and which species are present in each (helpful when you are iteratively adding new sequences to your database and want to see which species don't have genes representing them in a particular database)

Make it easier to specify whether you want to use the protein or nucleotide databases for fungi. Could add to this dictionary if we want to add other categories (Staph, bacteria, whatever)

In [ ]:
db_dict = {"fungus":{"nucl":"fungal_dbs_nucl", "prot":"fungal_dbs_prot"}}

Creates a dictionary of all the nucleotide and protein databases within the high-level db categories, to get a list of available genes that could be searched for

In [ ]:
quik_db = {}
for i in db_dict:
    quik_db[i] = {}
    for j in db_dict[i]:
        quik_db[i][j] = []
for i in db_dict:
    for j in db_dict[i]:
        db_folder = db_dict[i][j]
        for directory in glob.glob(db_folder+"/*"):
            gene = directory.split("/")[1]
            quik_db[i][j].append(gene)

Creates empty dictionary in which to save the species included in each dictionary

In [ ]:
dictionary_of_db_species  = {}
for i in db_dict:
    dictionary_of_db_species[i] = {}
    for j in db_dict[i]:
        dictionary_of_db_species[i][j] = {}

Iterates through all the blast databases to find species present in each

For databases of genes which contain that species, tag is "YES"

In [ ]:
import re
for i in db_dict:
    for j in db_dict[i]:
        db_folder = db_dict[i][j]
        gene_list = quik_db[i][j]
        for directory in glob.glob(db_folder+"/*"):
            gene = directory.split("/")[1]
            for file in glob.glob(directory+"/*.fasta"):
                for record in SeqIO.parse(file, "fasta"):
                    mrna = ""
                    if j=="nucl":
                        if re.search('mrna', record.description, re.IGNORECASE):
                            mrna = "_mRNA"
                        species = " ".join(record.description.split(" ")[1:3])
                    if j=="prot":
                        species = record.description.split("[")[1].strip("]")
                    if species in dictionary_of_db_species[i][j]:
                        dictionary_of_db_species[i][j][species][gene] = "YES"+mrna
                    else:
                        dictionary_of_db_species[i][j][species] = {i:"" for i in gene_list}
                        dictionary_of_db_species[i][j][species][gene] = "YES"

Creates new csv spreadsheet listing all the species in rows, and the blast databases (i.e. genes) in columns: presence/absence matrix

In [ ]:
for species_type in dictionary_of_db_species:
    for blast_type in dictionary_of_db_species[species_type]:
        header_list = quik_db[species_type][blast_type]
        term = species_type+"_"+blast_type
        outfile = open(term+"_database.csv", "w")
        outfile.write(","+",".join(header_list)+"\n")
        newlines = []
        for species in dictionary_of_db_species[species_type][blast_type]:
            info = [dictionary_of_db_species[species_type][blast_type][species][i] for i in header_list]
            newline = species+","+",".join(info)+"\n"
            newlines.append(newline)
        newlines.sort()
        for newline in newlines:
            outfile.write(newline)
        outfile.close()

## Inputs for the actual blast search

organism_type = fungus or bacteria

blast_type = "nucl" or "prot" (are you using the nucleotide or protein database)

update_blastdb = True or False <-- should the databases be remade (if you haven't added anything or otherwise touched them, this can be False)

input_jc_list = name of txt file containing list of jc numbers to be blasted

job_name = identifier which will be use to store results for the job (results will be in a directory under this name, so make it easy to identify, don't call it "results" or something)

filter_factor = what percent length of the reference sequence should the query alignment match? .9 = 90%, for example. Best to start out with this number as a low value (<70%) and see what matches you get, then blast those matches against the NCBI database to see what new sequences to add to the database.

NOTE: this script will automatically check for a directory called "jc_files", and 

In [ ]:
organism_type = "fungus"
blast_type = "nucl"
update_blastdb = True
input_jc_list = "test_file.txt"
job_name = "fungal_gene_pull_test"
filter_factor = .9
download_queries=True

## Code for blasting

### Prepare places to store results from inputs
- create new directory from job name
- create blast results folder within new directory
- get list of genes in fungal database
- create directories for pulled genes

In [ ]:
## Create new directory from job name
new_dir(job_name)
## Create blast results folder within new directory
new_dir(job_name+"/blast_results")
blast_results_dir = job_name+"/blast_results"
## Get directory containing all the dbs used for this job based on fungus or bacteria
db_to_use = db_dict[organism_type][blast_type]
## from proper db directory, get list of genes (i.e. subfolders)
genes = [directory.split("/")[-1] for directory in glob.glob(db_to_use+"/*")]
## Create empty dictionary for storing directory information for each gene
gene_blasting_dict = {}
gene_pulling_dict = {}
for i in genes:
    place_for_blast_results = blast_results_dir+"/"+i+"_blast_results"
    place_for_pulled_gene = job_name+"/"+i+"_pulled_gene"
    gene_blasting_dict[i] = place_for_blast_results
    gene_pulling_dict[i] = place_for_pulled_gene
    new_dir(place_for_blast_results)
    new_dir(place_for_pulled_gene)

### Make updated blast dbs (specify in inputs whether you want to update them or not, depending on whether you added new sequences)

In [ ]:
gene_to_db_dict = {}
for directory in glob.glob(db_to_use+"/*"):
    for file in glob.glob(directory+"/*.fasta"):
        gene_to_db_dict[directory.split("/")[-1]] = make_blast_db(file, blast_type, makenew=update_blastdb)

### Create dictionary linking each database sequence id to the species it represents and its length. 

This will be used below to calculate the % length of an alignment to the reference sequence, as well as to build the results spreadsheets.

In [ ]:
gene_to_species_and_len = {}
from Bio import SeqIO
for gene in gene_to_db_dict:
    gene_to_species_and_len[gene] = {}
    for record in SeqIO.parse(gene_to_db_dict[gene], "fasta"):
        if blast_type=="nucl":
            species = " ".join(record.description.split(" ")[1:3])
        if blast_type=="prot":
            species = record.description.split("[")[1].strip("]")
        gene_to_species_and_len[gene][record.id] = [species, len(record.seq)]

### Download needed jc files and save to dictionary (jc # : path to downloaded jc fasta file)

In [ ]:
if query_directory!="":
    new_dir(query_directory)
jc_dict = download_jcs_from_list(input_jc_list)

### Actual blasting

Iterate through each blast db that will be used for the job, and blast all the jc files against it. I built this for using on my laptop but for high-volume jobs on blastkid, should be able to assign each blast to its own core so that the jobs can be done in parallel.

In [ ]:
for gene in gene_blasting_dict:
    mydatabase = gene_to_db_dict[gene]
    blast_results_folder = gene_blasting_dict[gene]
    for jc_num in jc_dict:
        query = jc_dict[jc_num].strip("/")
        do_a_blast_to_folder(fasta_query=query, database=mydatabase, database_name=gene, database_type=blast_type, folder=blast_results_folder) 

### Parsing blast results to get best match for each jc file

The meat of the matter: deciding which is the best blast hit for each gene and pulling out that gene, for each combination of query genome and gene database. This is pretty basic but it seems to perform well.

This is not currently set up to detect multiple copies of the gene in a genome, just to find the best match.

Steps are as follows:
1. Iterates through each gene/blast database of interest (Example: calmodulin database)
2. Then iterates through each blast result for that gene
3. Returns "NO MATCH" if blast result file is empty
4. Otherwise, iterates through each match in the blast result file, with an initial 'best % identity' of 0
5. If a match doesn't meet the % requirement specified by 'filter_factor' in the inputs, it is skipped
6. Otherwise the % identity of the match is compared with the previous % identity, and if the % identity is better, it is saved as the 'best % identity'
7. Once all results from the blast results file have been compared, saves the best match to a dictionary
8. If none of the results in the blast results file met the length requirement, returns "NO MATCH MORE THAN (some % you specified) OF REF SEQ" instead of the match.

In [ ]:
results = {}
for gene in gene_blasting_dict:
    results[gene] = {}
    blast_saved_loc = gene_blasting_dict[gene]
    gene_save_loc = gene_pulling_dict[gene]
    for file in glob.glob(blast_saved_loc+"/*.blast*"):
        jc_num = find_JC_number(file)
        if os.stat(file).st_size == 0:
            results[gene][jc_num] = ["NO MATCH"]
        else:
            query = jc_dict[jc_num].strip("/")
            newfilename = gene_save_loc+"/"+jc_num+"_"+gene+".fasta"
            with open(file) as infile:
                best_perc_ident = 0
                best_db = "no good match"
                for line in infile:
                    cols = line.strip().split("\t")
                    db_name = cols[1]
                    perc_ident = cols[2]
                    alignment_length = cols[3]
                    ref_len_perc = float(gene_to_species_and_len[gene][db_name][1])*filter_factor
                    if ref_len_perc < float(alignment_length):
                        if float(perc_ident) > best_perc_ident:
                            best_perc_ident = float(perc_ident)
                            scaffold_name = cols[0]
                            best_db = db_name
                            qstart = int(cols[6])
                            qend = int(cols[7])
                            if qstart > qend:
                                temp = qstart
                                qstart = qend
                                qend = temp
                            match = [gene_to_species_and_len[gene][best_db][0], 
                                     perc_ident, scaffold_name, alignment_length, gene_to_species_and_len[gene][best_db][1],qstart, qend]
                if best_db == "no good match":
                    results[gene][jc_num] = ["NO MATCH MORE THAN {}% OF REF SEQ".format(int(filter_factor*100))]
                else:
                    results[gene][jc_num] = match
                    pull_a_sequence(query, match[2], match[5], match[6], newfilename)

### Get ordered list of jc numbers of this job for saving results to csv

In [ ]:
jc_list_for_saving = list(jc_dict.keys())
jc_list_for_saving.sort()

### Write full results for each gene to file and collect info for writing 

In [ ]:
gene_specific_header = ",".join(["JC_Number","Best_Match_Species","Percent_identity","Scaffold","Alignment_length","Reference_seq_length","Start","End"])+"\n"
general_results_header = ["JC_Number"]
general_results = {i:[] for i in jc_list_for_saving}
for gene in results:
    outfilename = job_name+"/"+job_name+"_"+gene+".csv"
    outfile = open(outfilename, "w")
    outfile.write(gene_specific_header)
    general_results_header = general_results_header + [gene+"_best_match_species", gene+"_perc_identity"]
    for jc_num in jc_list_for_saving:
        species = results[gene][jc_num][0]
        if "NO MATCH" in species:
            general_results[jc_num] = general_results[jc_num]+[species+","]
        else:
            general_results[jc_num] = general_results[jc_num]+[species,str(results[gene][jc_num][1])]
        newline = jc_num+","+",".join([str(i) for i in results[gene][jc_num]])+"\n"
        outfile.write(newline)
    outfile.close()

### Write general results for each gene to file

In [ ]:
with open(job_name+"/"+job_name+"_"+"results.csv", "w") as outfile:
    outfile.write(",".join(general_results_header)+"\n")
    for jc_num in jc_list_for_saving:
        write_statement = jc_num+","+",".join(general_results[jc_num])+"\n"
        outfile.write(write_statement)